# CHAPTER 10 인경신경망 소개

인경 신경망(Artificial neural networks, ANN)은 뇌의 구조에서 영감을 받은 알고리즘 종류이다. 뇌의 뉴런과 같이 무수히 많은 뉴런을 통해
사람처럼 복잡한 의사결정을 내릴 수 있다는 아이디어이다. 이를 통해서, 오늘날의 구글 검색엔진과 추천 알고리즘, 대표적으로 알파고 등이
존재한다. 설명을 위해 김성훈 교수님의 모두의 딥러닝을 사용하였다.

In [10]:
from IPython.display import Image

![](C:\Users\heon1\ml_handson\images\10\1.png)

![](./images/10/1.png)

![](./images/10/1.png)
뇌의 구조와 같이 정보를 받았을 때 정보를 취합하여, 일정치를 넘을 때 다음 
신경으로 넘긴다. 이는 아래와 같이 데이터를 취합하여 활성화 함수를 통해 값을
출력하는 것이다.


이러한 뉴런을 무수히 많이 여러층으로 논 것이 신경망이라고 볼 수 있다. 밑의 그림처럼 
간단한 아이디어 같지만, 무수히 많은 뉴런이 있을 때 뛰어난 결과를 보인다. 
![](./images/10/2.png)

![](./images/10/3.png)

신경망은 처음 발표시 단층의 경우 선형의 조합이었으므로, xor같은 비선형 문제를 
풀 수 없었고, 이러한 한계점으로 사용되지 않았다. 하지만 Layer를 한층 더
늘려 비선형성을 추가할 때, 이를 극복하여 가능하게 되었다.
![](./images/10/4.png)
우리는 다층 신경망의 대표적인 뉴럴넷 MultilayerPerceptron, MLP를 간단히 실습해 볼 것이다.

### 10.1.3 퍼셉트론

In [15]:
import os
import numpy as np
import tensorflow as tf

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
# matplotlib.rc('font', family='AppleGothic')  # MacOS
matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False
import numpy as np 
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2,3)]
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state = 42)
per_clf.fit(X,y)

y_pred = per_clf.predict([[2, 0.5]]) # sklearn에서 지원하지만 실제론 쓰이지 않는다.
print(y_pred)


C:\Users\heon1\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\heon1\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\heon1\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


[1]


### 텐서플로의 저수준 api로 심층 신경망 학습하기

가장 대표적인 신경망 데이터셋인 mnist를 통하여, 간단하게 mlp를 구현해보는 것을 
목표로 한다. 코드는 https://github.com/sjchoi86/Tensorflow-101/blob/master/notebooks/mlp_mnist_simple.ipynb
에서 가져왔다. 우리의 mlp구조는 아래와같다.
![신경망](Desktop\스터디공부\10장\5.png)

In [17]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings("ignore")
print("packages loaded")
print("tf version is%s" % (tf.__version__))

packages loaded
tf version is1.14.0


In [19]:
# mnist
mnist = input_data.read_data_sets('data/', one_hot=True)
mnist
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print("mnust loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
mnust loaded


In [20]:
print(mnist)
print(trainimg.shape)
print(trainlabel.shape)

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000021A953F5FC8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000021A96ECBF88>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000021A96ECBE88>)
(55000, 784)
(55000, 10)


In [21]:
# NETWORK TOPOLOGIES
n_hideen_1 = 256
n_hideen_2 = 128
n_input = 784
n_classes = 10
epochs = 50
batch_size = 100
display_step = 10
# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [23]:
#NETWORK PARAMETERS
stddev = 0.1
weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hideen_1], stddev=stddev)),
    'h2' : tf.Variable(tf.random_normal([n_hideen_1, n_hideen_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hideen_2, n_classes], stddev=stddev))
}
biases = {
    'b1' : tf.Variable(tf.random_normal([n_hideen_1])),
    'b2' : tf.Variable(tf.random_normal([n_hideen_2])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [24]:
#MLP AS FUNCTION
def multilayer_perceptron(_X, _weights, _biases) :
    layer1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1']))
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2']))
    return (tf.matmul(layer2, _weights['out'])+ _biases['out'])

In [25]:
pred = multilayer_perceptron(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits= pred))
optm = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(cost)
corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [30]:
# PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 0.055342026
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.963
Epoch: 007/020 cost: 0.038997449
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.967
Epoch: 011/020 cost: 0.030459012
TRAIN ACCURACY: 0.980
TEST ACCURACY: 0.970
Epoch: 015/020 cost: 0.029607711
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.970
Epoch: 019/020 cost: 0.019694811
TRAIN ACCURACY: 0.960
TEST ACCURACY: 0.970
OPTIMIZATION FINISHED


### 하이퍼 파라미터 조정

딥러닝은 많은 하이퍼파라미터를 가지고 있으며, 이들의 조정을 통해
학습 성과와 속도의 개선을 가져올 수 있다. 간단히 앞선 모델을 수정하기 위해서
그림에서 설명하면 사용되었던 relu를 sigmoid function으로 변경할 수 있다.
추가로 비선형성의 추가를 위해 layer를 늘리고 파라미터 수도 자연스럽게 늘어났다.
물론, 이러한 수정은 학습의 성과를 낮출 가능성이 크므로, 상황에 따라 조정하는 것이 적절하다.
![](./images/10/6.png)

추가로 사용된 하이퍼 PARAMETERS 중

training_epochs = 20

batch_size      = 100

display_step    = 4

optm = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(cost)

optimizer를 adam이 아닌 momentum과 같은 다른 알고리즘으로 변환 할 수 있으며,
dropout과 같은 기법을 추가하는 등 다양한 방법이 존재한다.